# Run Gradient Optimisation for Feature Selection on Water Dataset

In [2]:
import pathlib
import re

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from ase.io import read as ase_read

from dadapy.feature_weighting import FeatureWeighting

## Import Data

Large SOAP datasets and especially ACSFs are somewhat slow to calculate, so I mostly just run this on our local cluster and then load from numpy files.
To recalculate the descriptors, use `make_descriptors.py`.

In [3]:
data_dir = pathlib.Path("../data").resolve()

In [ ]:
# Get ase.Atoms objects for each liquid configuration
liquid_frames = ase_read(data_dir.joinpath("ice_in_water_data/dataset_1000_eVAng.xyz"), index=':')
n_atoms = np.sum(np.asarray([len(frame) for frame in liquid_frames], dtype=np.int16))
atom_types = np.zeros((n_atoms), dtype=np.int8)

# Collect some metadata, like how many atoms/config, atoms in total and which atom is even an oxygen.
counter = 0
for frame in liquid_frames:
    atom_types[counter:counter+len(frame)] = frame.get_atomic_numbers()
    counter+=len(frame)
is_o = atom_types==8
is_h = np.logical_not(is_o)

print(f"Found {np.count_nonzero(is_o)} Oxygen atoms and {np.count_nonzero(is_h)} Hydrogen atoms.")

In [ ]:
# Get new descriptors from file or laod ase.Atoms objects and recalculate
# Recalculation here works best when just getting SOAPs, ACSF takes a little long
average_soap = np.load(data_dir.joinpath("ice_in_water_data/average_soap_rcut6_nmax6_lmax6_sigma03.npy"))
atomic_soap = np.load(data_dir.joinpath("ice_in_water_data/singleatom_soap_rcut6_nmax6_lmax6_sigma03.npy"))
print("Fetched computed atomic SOAP descriptors for %u configurations and with %u features each."%atomic_soap.shape)
print("Fetched computed global SOAP descriptors for %u configurations and with %u features each."%average_soap.shape)

# The file format of the input file the descriptors are calculated from is 54 solid, 1000 liquid
# So we can just get the liquid configurations by getting the number of atoms n_atoms in the liquid configurations
# From the end of the decriptor matrix
liquid_atomic_soap = atomic_soap[-n_atoms:, :].copy()

In [ ]:
average_acsf = np.asarray(np.load(data_dir.joinpath("ice_in_water_data/average_acsf_rcut6_gridsearch_bohr_lambda.npy")), dtype=np.float32)
atomic_acsf = np.asarray(np.load(data_dir.joinpath("ice_in_water_data/singleatom_acsf_rcut6_gridsearch_bohr_lambda.npy")), dtype=np.float32)
liquid_atomic_acsf = atomic_acsf[-n_atoms:, :].copy()
print("Fetched computed atomic SOAP descriptors for %u configurations and with %u features each."%atomic_acsf.shape)
print("Fetched computed liquid atomic SOAP descriptors for %u configurations and with %u features each."%liquid_atomic_acsf.shape)
print("Fetched computed global SOAP descriptors for %u configurations and with %u features each."%average_acsf.shape)

### Post Processing of Input Data

In [ ]:
descriptors = [average_soap, atomic_soap, liquid_atomic_soap, average_acsf, atomic_acsf, liquid_atomic_acsf]
for desc in descriptors:
    desc /= np.linalg.norm(desc, axis=-1)[:, np.newaxis]
average_soap, atomic_soap, liquid_atomic_soap, average_acsf, atomic_acsf, liquid_atomic_acsf = descriptors

# apparently atomic acsf sometimes become nan, set to 0
nan_frames = np.argwhere(np.isnan(atomic_acsf))[:, 0]
print("Removing %u nan frames in atomic acsf"%(len(np.unique(nan_frames))))
atomic_acsf[nan_frames, :] = 0.

nan_frames = np.argwhere(np.isnan(liquid_atomic_acsf))[:, 0]
print("Removing %u nan frames in liquid atomic acsf"%(len(np.unique(nan_frames))))
liquid_atomic_acsf[nan_frames, :] = 0.

### Defining Input and Target Data

`target_data` is the space which delivers the target ranks, `input_space` is the space for which the gammas should be optimised.

In [ ]:
rng = np.random.default_rng()
# random_selection = rng.choice(liquid_atomic_soap.shape[0], 300)
target_data = liquid_atomic_soap[::500]
input_space = liquid_atomic_acsf[::500]

fw_input = FeatureWeighting(coordinates=input_space, maxk=input_space.shape[0]-1, verbose=True)
fw_target = FeatureWeighting(coordinates=target_data, maxk=target_data.shape[0]-1, verbose=True)

if True:
    stds = np.std(input_space, axis=0)
    stds[stds==0.] = 1.
    standardised_input = input_space/stds[np.newaxis, :]
    # dist_matrix = kimb.compute_dist_matrix(data=target_data.astype(np.double), period=np.zeros((target_data.shape[-1])))
    # truth_ranks = stats.rankdata(dist_matrix, method='average', axis=1).astype(int, copy=False)

print(f"Working on ground truth shaped {target_data.shape} and optimising space of shape {input_space.shape}")

## Lasso Optimisation

### Find Optimal Learning Rate

In [ ]:
initial_gammas = np.ones((input_space.shape[-1],), dtype=np.double)
lr_list = np.logspace(-2, 2, 10)

opt_l_rate = fw_input.return_optimal_learning_rate(
    target_data=fw_target, initial_weights=initial_gammas, lambd=None, 
    n_epochs=50, decaying_lr=True, 
    n_samples=154, trial_learning_rates=lr_list,
)

kernel_imbalances_list = fw_input.history['dii_per_epoch_per_lr'][np.argwhere(opt_l_rate==lr_list)[0, 0], :]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.plot(kernel_imbalances_list, label="kernel imbalances")

ax.set_title("Loss for optimal learning rate %f"%opt_l_rate)
# ax.set_yscale('log')
ax.set_ylabel('Kernel Imbalance')
ax.set_xlabel("epoch")

plt.show()

### Optimise using Lasso Regularisation

In [ ]:
(
    num_nonzero_features,
    l1_penalties_opt_per_nfeatures,
    dii_opt_per_nfeatures, 
    weights_opt_per_nfeatures
) = fw_input.return_lasso_optimization_dii_search(
    target_data=fw_target, initial_weights=initial_gammas, lambd=None, 
    learning_rate=opt_l_rate, constrain=True, decaying_lr=True,
    n_epochs=100, refine=True
)

### Post Processing Gammas from Lasso Optimisation

The weights returned by lasso optimisation are returned unordered.
This recalculates the information imbalance for each entry and orders the weights by number of non-zero features and best information imbalance.

In [ ]:
# invert order so it has least features first
kernel_imbs = dii_opt_per_nfeatures[::-1]
lasso_gammas = weights_opt_per_nfeatures[::-1]
num_nonzero_features = num_nonzero_features[::-1]
l1_penalties_opt_per_nfeatures = l1_penalties_opt_per_nfeatures[::-1]

## Classic Backwards Greedy Feature Selection

In [ ]:
# For ndim_red times, find the optimal representation
ndims = input_space.shape[-1]
delete_indices = []
classic_greedy_imbalances = np.zeros((ndims-1,), dtype=np.float32)
for ndim in range(ndims-1):
    print("Removing %u features out of %u"%(ndim, ndims))
    information_imbalances = []

    # Add kd-trees for current best guess appended with each new candidate
    candidates = np.setdiff1d(np.arange(standardised_input.shape[-1]), delete_indices)
    for candidate in candidates:
        candidate_representation = np.delete(standardised_input, delete_indices+[candidate], axis=-1)
        information_imbalances.append(get_lambdaopt_kimb(candidate_representation, truth_ranks))

    best_cand_ind = np.argmin(np.asarray(information_imbalances))
    delete_indices.append(candidates[best_cand_ind])
    classic_greedy_imbalances[ndim] = get_lambdaopt_kimb(
        np.delete(standardised_input, delete_indices, axis=-1),
        truth_ranks
    )
classic_delete_indices = np.asarray(delete_indices, dtype=np.int16)

## Weighted Backwards Greedy

In [ ]:
greedy_gammas, greedy_imbs = fw_input.return_backward_greedy_dii_elimination(
    target_data=fw_target, initial_weights=initial_gammas, lambd=None,
    initial_weights=initial_gammas, lambd=None, n_epochs=80,
    l_rate=opt_l_rate, constrain=False, decaying_lr=True
)

## Evaluations and Plotting

If no recalculations are needed just go to this part and run it.

In [9]:
mode='load_old'
if mode=='save_new':
    np.save(data_dir.joinpath('water_phase_store/kernel_imbs_hartbohr_lambda.npy'), kernel_imbs)
    np.save(data_dir.joinpath('water_phase_store/lasso_gammas_hartbohr_lambda.npy'), lasso_gammas)
    np.save(data_dir.joinpath('water_phase_store/classic_greedy_imbalances_hartbohr_lambda.npy'), classic_greedy_imbalances)
    np.save(data_dir.joinpath('water_phase_store/classic_delete_indices_hartbohr_lambda.npy'), classic_delete_indices)
    np.save(data_dir.joinpath("water_phase_store/greedy_gammas_hartbohr_lambda.npy"), greedy_gammas)
    np.save(data_dir.joinpath("water_phase_store/greedy_imbs_hartbohr_lambda.npy"), greedy_imbs)
elif mode=='load_old':
    kernel_imbs = np.load(data_dir.joinpath('water_phase_store/kernel_imbs_hartbohr_lambda.npy'))
    lasso_gammas = np.load(data_dir.joinpath('water_phase_store/lasso_gammas_hartbohr_lambda.npy'))
    classic_greedy_imbalances = np.load(data_dir.joinpath('water_phase_store/classic_greedy_imbalances_hartbohr_lambda.npy'))
    classic_delete_indices = np.load(data_dir.joinpath('water_phase_store/classic_delete_indices_hartbohr_lambda.npy'))
    greedy_gammas = np.load(data_dir.joinpath("water_phase_store/greedy_gammas_hartbohr_lambda.npy"))
    greedy_imbs = np.load(data_dir.joinpath("water_phase_store/greedy_imbs_hartbohr_lambda.npy"))

where_gammas_found = (kernel_imbs!=0.)

In [ ]:
n_batches = 10
rng = np.random.default_rng()
batch_indices = rng.choice(liquid_atomic_soap.shape[0], size=(n_batches, input_space.shape[0]), replace=False)

true_fws = []
for batch in batch_indices:
    true_fws.append(FeatureWeighting(
        coordinates=liquid_atomic_soap[batch, :], maxk=len(batch)-1, verbose=False
    ))

print("Kernel Imbalance")
batched_lasso_imbalances = np.empty((len(kernel_imbs), n_batches), dtype=np.float32)
for n_dims in range(len(kernel_imbs)):
    if where_gammas_found[n_dims]:
        for ii_batch, batch in enumerate(batch_indices):
            batched_lasso_imbalances[n_dims, ii_batch] = FeatureWeighting(
                coordinates=liquid_atomic_acsf[batch, :]*lasso_gammas[n_dims, :], maxk=len(batch)-1, verbose=False
            ).return_dii(
                target_data=true_fws[ii_batch],
                lambd=None
            )

print("Classic Greedy Imbalance")
batched_classic_imbalances = np.empty((len(classic_greedy_imbalances), n_batches), dtype=np.float32)
for n_dims in range(len(classic_delete_indices)):
    for ii_batch, batch in enumerate(batch_indices):
        batched_classic_imbalances[n_dims, ii_batch] = FeatureWeighting(
            coordinates=np.delete(liquid_atomic_acsf[batch, :]/stds[np.newaxis, :], classic_delete_indices[:n_dims+1], axis=-1),
            maxk=len(batch)-1, verbose=False,
        ).return_dii(
            target_data=true_fws[ii_batch],
            lambd=None
        )

print("Greedy Imbalance")
batched_greedy_imbalances = np.empty((len(greedy_imbs), n_batches), dtype=np.float32)
for n_dims in range(greedy_gammas.shape[-1]):
    for ii_batch, batch in enumerate(batch_indices):
        batched_greedy_imbalances[n_dims, ii_batch] = FeatureWeighting(
            coordinates=liquid_atomic_acsf[batch, :]*greedy_gammas[n_dims, -1, :], 
            maxk=len(batch)-1, verbose=False,
        ).return_dii(
            target_data=true_fws[ii_batch],
            lambd=None
        )

print("Greedy only Select")
greedy_select_imbalance = np.empty((greedy_gammas.shape[-1], ), dtype=np.float32)
for n_dims in range(greedy_gammas.shape[-1]):
    greedy_select_imbalance[n_dims] = FeatureWeighting(
        coordinates=(input_space/stds[np.newaxis, :])[:, greedy_gammas[n_dims, -1, :]!=0],
        maxk=len(batch)-1, verbose=False,
    ).return_dii(
        target_data=fw_target,
        lambd=None
    )

In [ ]:
print("Greedy only Select")
greedy_select_imbalance = np.empty((greedy_gammas.shape[-1], ), dtype=np.float32)
for n_dims in range(greedy_gammas.shape[-1]):
    greedy_select_imbalance[n_dims] = FeatureWeighting(
        coordinates=(input_space/stds[np.newaxis, :])[:, greedy_gammas[n_dims, -1, :]!=0], 
        maxk=len(batch)-1, verbose=False,
    ).return_dii(
        target_data=fw_target,
        lambd=None
    )

print("Random Selection")
batched_random_imbalances = np.empty((len(kernel_imbs), n_batches), dtype=np.float32)
rand_gamma_mask = np.zeros((input_space.shape[-1], ), dtype=bool)
for n_dims in range(1, len(kernel_imbs)):
    for ii_batch, batch in enumerate(batch_indices):
        rand_gamma_mask[:] = False
        rand_gamma_mask[rng.choice(input_space.shape[-1], size=(n_dims, ), replace=False)] = True
        batched_random_imbalances[n_dims, ii_batch] = FeatureWeighting(
            coordinates=(liquid_atomic_acsf[batch, :]/stds)[:, rand_gamma_mask], 
            maxk=len(batch)-1, verbose=False,
        ).return_dii(
            target_data=true_fws[ii_batch],
            lambd=None
        )

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

where_gammas_found = (kernel_imbs!=0.)
fig, ax = plt.subplots(1, 1, figsize=(8, 3))

if False:
    ax.plot(range(1, len(classic_greedy_imbalances)+1), classic_greedy_imbalances[::-1], 'x--', label='classic greedy')
    ax.fill_between(
        range(1, len(classic_greedy_imbalances)+1), 
        np.min(batched_classic_imbalances[::-1, :], axis=1),
        np.max(batched_classic_imbalances[::-1, :], axis=1),
        alpha=0.3, linewidth=10
    )
    ax.plot(
    np.arange(1, greedy_select_imbalance.shape[0]+1), greedy_select_imbalance[::-1],
    'o--', label='greedy select'
    )

ax.plot(
    np.arange(kernel_imbs.shape[0])[where_gammas_found], kernel_imbs[where_gammas_found],
    'o--', label='lasso optimisation'
)
ax.fill_between(
    np.arange(kernel_imbs.shape[0])[where_gammas_found], 
    np.min(batched_lasso_imbalances[where_gammas_found, :], axis=1),
    np.max(batched_lasso_imbalances[where_gammas_found, :], axis=1),
    alpha=0.3
)

ax.plot(
    np.arange(1, greedy_imbs.shape[0]+1), greedy_imbs[::-1, -1],
    'o--', label='greedy weighted optimisation'
)
ax.fill_between(
    range(1, len(batched_greedy_imbalances)+1), 
    np.min(batched_greedy_imbalances[::-1, :], axis=1),
    np.max(batched_greedy_imbalances[::-1, :], axis=1),
    alpha=0.3, linewidth=10
)

ax.vlines(
    np.arange(1, kernel_imbs.shape[0]+1), 
    np.min(batched_random_imbalances, axis=1),
    np.max(batched_random_imbalances, axis=1),
    alpha=0.8, color='tab:grey', linewidth=5, label='random selection'
)

ax.set_xlabel('Dimensions')
ax.set_ylabel(r'$DII(S^{red} \to S^{true})$')
ax.set_title("Comparison of %u ACSF descriptors to %u SOAP descriptors\nfor %u different atom environments"%(input_space.shape[1], target_data.shape[-1], input_space.shape[0]))
ax.legend()
ax.set_ylim([0., 0.6])
ax.set_xlim([1, 30])

fig.savefig("ii_history_acsf%u_c%u.png"%input_space.shape[::-1], format='png', bbox_inches='tight', dpi=300)
plt.show()

## Final Figure

In [ ]:
from glob import glob
import re

target_dir = data_dir.joinpath("n2p2_fitting/240930_pots_errorbars_out_predict/")

results_dict = {}
for logfile in target_dir.glob('*.log'):
    with open(logfile, 'r') as f:
        content = f.read()
        runtype = re.search("231213_pot_acsf(.*)_hartbohr_scaleunits_bcdata_lambda", content).group(1)
        run_num = re.search("run_([0-9]*)", content).group(1)

        # memory_kb = re.search("\s([0-9]*)maxresident", content).group(1)
        runtime_s = re.search("user\s0m([0-9]*\.[0-9]*)", content).group(1)
        
        pot_dir = target_dir.joinpath("231213_pot_acsf"+runtype+"_hartbohr_scaleunits_bcdata_lambda/run_"+run_num)

        all_errors = np.loadtxt(pot_dir.joinpath('learning-curve.out'))
        all_errors[:, 1:9] *= 27.211386245988
        all_errors[:, 9:13] *= 51.421 # Hartree/Bohr to eV/Angstrom conversion
        rmse_ftest_evA = all_errors[-1, 12]
        
        if runtype not in results_dict:
            results_dict[runtype] = {"runtimes": [], "rmses": []}
        
        results_dict[runtype]["runtimes"].append(float(runtime_s))
        results_dict[runtype]["rmses"].append(rmse_ftest_evA) 
print(results_dict)

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.ticker import StrMethodFormatter

plt.rcParams.update(plt.rcParamsDefault)
fontsize = 12

if False:
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "Helvetica",
        "font.size": fontsize,
        'text.latex.preamble': r'\usepackage{amsmath}'
    })

double_ax = False

fig, axes = plt.subplots(2, 1, figsize=(6, 6))

kernel_keys = ['10', '18', '25', '38', '50', '176']
rand_keys = ['10rand', '18rand', '25rand', '38rand', '50rand', '176']

cmap = plt.get_cmap('viridis')


rcolor = "#3f4e32" # cmap(0.1)
lcolor = "#6a272a" # cmap(0.9)
greedy_dii_color = "#f29944" # cmap(0.6)
lasso_dii_color = "#989ec1" # cmap(0.3)
random_dii_color = "tab:grey"

mae_ax = axes[1]

mae_ax.plot(
    [int(kernel_key) for kernel_key in kernel_keys],
    [np.mean(results_dict[kernel_key]["rmses"]) for kernel_key in kernel_keys],
    label=r'L$_1$ reg.', color=lcolor, marker='o', linestyle='-'
)
mae_ax.plot(
    [int(kernel_key) for kernel_key in kernel_keys],
    [np.mean(results_dict[rand_keys]["rmses"]) for rand_keys in rand_keys],
    label='random', color=lcolor, marker='^', linestyle='-.'
)
# mae_ax.plot(176, results_dict['176']["rmse"], color='tab:green', marker='o', markersize=4)
mae_ax.set_xlabel("Number of non-zero features", fontsize=fontsize)

mae_ax.set_ylabel("Test Force RMSE [eV/A]", color=lcolor, fontsize=fontsize)
mae_ax.grid(axis='y', color=lcolor)
mae_ax.tick_params(axis='x', which='both', top=True, labeltop=False, labelsize=fontsize)
if double_ax:
    mae_ax.tick_params(axis='y', which='both', color=lcolor, labelcolor=lcolor, labelsize=fontsize)
else:
    mae_ax.tick_params(axis='y', which='both', labelsize=fontsize)
# mae_ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(4))
# axes[0].set_xscale('log')
# axes[0].set_yscale('log')

time_ax = mae_ax.twinx()

time_ax.plot(
    [int(kernel_key) for kernel_key in kernel_keys], 
    [np.mean(results_dict[kernel_key]["runtimes"]) for kernel_key in kernel_keys],
    label=r'L$_1$ reg.', color=rcolor, marker='o', linestyle='-'
)
time_ax.plot(
    [int(kernel_key) for kernel_key in kernel_keys], 
    [np.mean(results_dict[rand_keys]["runtimes"]) for rand_keys in rand_keys],
    label='random', color=rcolor, marker='^', linestyle='-.'
)
# time_ax.plot(176, results_dict['176']["runtime"], color='tab:green', marker='o', markersize=4)
time_ax.set_ylabel("Runtime [au]", color=rcolor, fontsize=fontsize)
time_ax.yaxis.set_label_position("right")
time_ax.tick_params(axis='x', which='both', bottom=False, labelbottom=False, labelsize=fontsize)
time_ax.tick_params(axis='y', which='both', left=False, labelleft=False, right=True, labelright=True, color=rcolor, labelcolor=rcolor, labelsize=fontsize)
# time_ax.grid(axis='y', color=rcolor)
# time_ax.yaxis.set_major_locator(mpl.ticker.LinearLocator(4))

# time_ax.set_ylim((0, 75))
# mae_ax.set_ylim((0, 0.3))

dii_ax = axes[0]

dii_ax.plot(
    np.arange(1, greedy_imbs.shape[0]+1), greedy_imbs[::-1, -1],
    'D--', label='greedy weighted optimisation', color=greedy_dii_color
)
dii_ax.vlines(
    range(1, len(batched_greedy_imbalances)+1), 
    np.min(batched_greedy_imbalances[::-1, :], axis=1),
    np.max(batched_greedy_imbalances[::-1, :], axis=1),
    alpha=1, color=greedy_dii_color, linewidth=3
)

dii_ax.plot(
    np.arange(kernel_imbs.shape[0])[where_gammas_found], kernel_imbs[where_gammas_found],
    'o-', label='lasso optimisation', color=lasso_dii_color
)
dii_ax.vlines(
    np.arange(kernel_imbs.shape[0])[where_gammas_found], 
    np.min(batched_lasso_imbalances[where_gammas_found, :], axis=1),
    np.max(batched_lasso_imbalances[where_gammas_found, :], axis=1),
    alpha=1, color=lasso_dii_color, linewidth=3
)
dii_ax.vlines(
    np.arange(1, kernel_imbs.shape[0]+1), 
    np.min(batched_random_imbalances, axis=1),
    np.max(batched_random_imbalances, axis=1),
    alpha=0.8, color=random_dii_color, linewidth=3
)

# dii_ax.set_xlabel('Number of non-zero features', fontsize=fontsize)
dii_ax.set_ylabel("DII", fontsize=fontsize)
dii_ax.tick_params(
    axis='x', which='both', labelsize=fontsize, 
    bottom=True, labelbottom=True, top=False, labeltop=False
)
dii_ax.tick_params(axis='y', which='both', labelsize=fontsize)
# dii_ax.legend(fontsize=fontsize)
dii_ax.set_ylim([0.05, 0.5])
dii_ax.set_xlim([1, 30])

time_ax.legend(
    handles=[
        Line2D([0], [0], marker='o', linestyle='-', color='k', label=r'L$_1$ reg.', alpha=1),
        Line2D([0], [0], marker='^', linestyle='-.', color='k', label='random', alpha=1),
    ],
    ncols=3, loc=(0.15, 0.85), columnspacing=0.5, handletextpad=0.3, handlelength=2.5,
    fontsize=fontsize, 
)

dii_ax.legend(
    handles=[
        dii_ax.errorbar([-1], [-1], yerr=0, marker='o', elinewidth=3, linestyle='-', color=lasso_dii_color, label=r'L$_1$ reg.'),
        dii_ax.errorbar([-1], [-1], yerr=0, marker='D', elinewidth=3, linestyle='--', color=greedy_dii_color, label='greedy'),
        dii_ax.errorbar([-1], [-1], yerr=0, marker='', elinewidth=3, linestyle='', color=random_dii_color, label='random', alpha=0.8),
    ],
    ncols=3, columnspacing=0.5, handletextpad=0.3, handlelength=2.5,
    fontsize=fontsize
)

mae_ax.xaxis.set_major_formatter(StrMethodFormatter("{x:.0f}"))
mae_ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.2f}"))
time_ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.0f}"))

dii_ax.xaxis.set_major_formatter(StrMethodFormatter("{x:.0f}"))
dii_ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.1f}"))
plt.tight_layout()

fig.savefig("waterphase_final.png", format='png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.ticker import StrMethodFormatter

plt.rcParams.update(plt.rcParamsDefault)
fontsize = 12

if False:
    plt.rcParams.update({
        "text.usetex": True,
        "font.family": "Helvetica",
        "font.size": fontsize,
        'text.latex.preamble': r'\usepackage{amsmath}'
    })

double_ax = False

fig, axes = plt.subplots(3, 1, figsize=(5, 6))

kernel_keys = ['10', '18', '25', '38', '50', '176']
rand_keys = ['10rand', '18rand', '25rand', '38rand', '50rand', '176']

cmap = plt.get_cmap('viridis')

greedy_dii_color = "#f29944" # cmap(0.6)
lasso_dii_color = "#989ec1" # cmap(0.3)
random_dii_color = "tab:grey"

mae_ax = axes[1]

mae_ax.plot(
    [int(kernel_key) for kernel_key in kernel_keys],
    [np.mean(results_dict[kernel_key]["rmses"]) for kernel_key in kernel_keys],
    label=r'L$_1$ reg.', color=lasso_dii_color, marker='o', linestyle='-'
)
mae_ax.fill_between(
    [int(kernel_key) for kernel_key in kernel_keys],
    [np.min(results_dict[kernel_key]["rmses"]) for kernel_key in kernel_keys],
    [np.max(results_dict[kernel_key]["rmses"]) for kernel_key in kernel_keys],
    alpha=0.4, color=lasso_dii_color
)
mae_ax.plot(
    [int(kernel_key) for kernel_key in kernel_keys],
    [np.mean(results_dict[rand_keys]["rmses"]) for rand_keys in rand_keys],
    label='random', color=random_dii_color, marker='^', linestyle='-.', alpha=0.8
)
mae_ax.fill_between(
    [int(kernel_key) for kernel_key in kernel_keys],
    [np.min(results_dict[rand_keys]["rmses"]) for rand_keys in rand_keys],
    [np.max(results_dict[rand_keys]["rmses"]) for rand_keys in rand_keys],
    alpha=0.2, color=random_dii_color
)
mae_ax.set_xlabel("Number of non-zero features", fontsize=fontsize)
mae_ax.set_ylabel("Test Force RMSE [eV/A]", fontsize=fontsize)
mae_ax.tick_params(axis='x', which='both', labelsize=fontsize)
mae_ax.tick_params(axis='y', which='both', labelsize=fontsize)
mae_ax.grid(axis='y')
mae_ax.legend(fontsize=fontsize)

time_ax = axes[2]

kernel_times = np.array([np.mean(results_dict[kernel_key]["runtimes"]) for kernel_key in kernel_keys])
kernel_uppers = np.array([np.max(results_dict[kernel_key]["runtimes"]) for kernel_key in kernel_keys])
kernel_lowers = np.array([np.min(results_dict[kernel_key]["runtimes"]) for kernel_key in kernel_keys])
random_times = np.array([np.mean(results_dict[rand_keys]["runtimes"]) for rand_keys in rand_keys])
random_uppers = np.array([np.max(results_dict[rand_keys]["runtimes"]) for rand_keys in rand_keys])
random_lowers = np.array([np.min(results_dict[rand_keys]["runtimes"]) for rand_keys in rand_keys])

max_time = np.max(np.concatenate([kernel_times, random_times]))
time_ax.plot(
    [int(kernel_key) for kernel_key in kernel_keys], 
    kernel_times/max_time,
    label=r'L$_1$ reg.', color=lasso_dii_color, marker='o', linestyle='-'
)
time_ax.plot(
    [int(kernel_key) for kernel_key in kernel_keys], 
    random_times/max_time,
    label='random', color=random_dii_color, marker='^', linestyle='-.', alpha=0.8
)
time_ax.fill_between(
    [int(kernel_key) for kernel_key in kernel_keys],
    kernel_lowers/max_time,
    kernel_uppers/max_time,
    alpha=0.4, color=lasso_dii_color
)
time_ax.fill_between(
    [int(kernel_key) for kernel_key in kernel_keys],
    random_lowers/max_time,
    random_uppers/max_time,
    alpha=0.2, color=random_dii_color
)
time_ax.set_xlabel("Number of non-zero features", fontsize=fontsize)
time_ax.set_ylabel("Runtime [au]", fontsize=fontsize)
time_ax.tick_params(axis='x', which='both', labelsize=fontsize)
time_ax.tick_params(axis='y', which='both', left=True, labelleft=True, labelsize=fontsize)
time_ax.grid(axis='y')
time_ax.legend(fontsize=fontsize)

dii_ax = axes[0]

dii_ax.plot(
    np.arange(1, greedy_imbs.shape[0]+1), greedy_imbs[::-1, -1],
    'D--', label='greedy weighted optimisation', color=greedy_dii_color
)
dii_ax.vlines(
    range(1, len(batched_greedy_imbalances)+1), 
    np.min(batched_greedy_imbalances[::-1, :], axis=1),
    np.max(batched_greedy_imbalances[::-1, :], axis=1),
    alpha=1, color=greedy_dii_color, linewidth=3
)

dii_ax.plot(
    np.arange(kernel_imbs.shape[0])[where_gammas_found], kernel_imbs[where_gammas_found],
    'o-', label='lasso optimisation', color=lasso_dii_color
)
dii_ax.vlines(
    np.arange(kernel_imbs.shape[0])[where_gammas_found], 
    np.min(batched_lasso_imbalances[where_gammas_found, :], axis=1),
    np.max(batched_lasso_imbalances[where_gammas_found, :], axis=1),
    alpha=1, color=lasso_dii_color, linewidth=3
)
dii_ax.vlines(
    np.arange(1, kernel_imbs.shape[0]+1), 
    np.min(batched_random_imbalances, axis=1),
    np.max(batched_random_imbalances, axis=1),
    alpha=0.8, color=random_dii_color, linewidth=3
)

# dii_ax.set_xlabel('Number of non-zero features', fontsize=fontsize)
dii_ax.set_ylabel("DII", fontsize=fontsize)
dii_ax.tick_params(
    axis='x', which='both', labelsize=fontsize, 
    bottom=True, labelbottom=True, top=False, labeltop=False
)
dii_ax.tick_params(axis='y', which='both', labelsize=fontsize)
dii_ax.set_xlabel("Number of non-zero features", fontsize=fontsize)
dii_ax.set_ylim([0.05, 0.5])
dii_ax.set_xlim([1, 30])

dii_ax.legend(
    handles=[
        dii_ax.errorbar([-1], [-1], yerr=0, marker='o', elinewidth=3, linestyle='-', color=lasso_dii_color, label=r'L$_1$ reg.'),
        dii_ax.errorbar([-1], [-1], yerr=0, marker='D', elinewidth=3, linestyle='--', color=greedy_dii_color, label='greedy'),
        dii_ax.errorbar([-1], [-1], yerr=0, marker='', elinewidth=3, linestyle='', color=random_dii_color, label='random', alpha=0.8),
    ],
    ncols=3, columnspacing=0.5, handletextpad=0.3, handlelength=2.5,
    fontsize=fontsize
)

mae_ax.xaxis.set_major_formatter(StrMethodFormatter("{x:.0f}"))
mae_ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.2f}"))
time_ax.xaxis.set_major_formatter(StrMethodFormatter("{x:.0f}"))
time_ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.1f}"))

dii_ax.xaxis.set_major_formatter(StrMethodFormatter("{x:.0f}"))
dii_ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.1f}"))
plt.tight_layout()

fig.savefig("waterphase_final_stacked.png", format='png', bbox_inches='tight', dpi=300)
plt.show()

### Reading out which ACSF are selected

In [ ]:
# All this does is give labels for ACSF descriptors, since acsf seems to be missing this functionality
acsf_g2 = np.loadtxt(data_dir.joinpath('ice_in_water_data/g2_params_gridsearch_bohr_lambda.txt'))
acsf_g4 = np.loadtxt(data_dir.joinpath('ice_in_water_data/g4_params_gridsearch_bohr_lambda.txt'))
acsf_species = [1, 8]
acsf_symbols = ['H', 'O']
acsf_labels = []

mask = [True]*len(lasso_gammas[0])
counter = 0
for symbol in acsf_symbols:
    counter+=1
    acsf_labels.append(f"G1 to {symbol}")
    for g2_param in acsf_g2:
        counter+=1
        acsf_labels.append(f"G2 to {symbol}, R0={g2_param[1]:.3E}, eta={g2_param[0]*((1./1.8897259886)**2.):.3E}")
for ii_spec1, species1 in enumerate(acsf_species):
    for ii_spec2, species2 in enumerate(acsf_species):
        if species2 >= species1:
            for g4_param in acsf_g4:
                if g4_param[2] == 0.:
                    mask[counter] = False
                counter+=1
                acsf_labels.append(f"G4 with species {acsf_symbols[ii_spec1]} and {acsf_symbols[ii_spec2]}, eta={g4_param[0]*((1./1.8897259886)**2.):.3E}, zeta={g4_param[1]:.3E}, lambda={g4_param[2]:.3E}")